In [1]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()



0

In [3]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET



In [4]:
processed_data_dir_path = "/workspace/data_processed"

In [5]:
import glob 
data_open_slr_paths = glob.glob("/workspace/data/external/*")

In [6]:
data_open_slr_paths

['/workspace/data/external/13',
 '/workspace/data/external/2',
 '/workspace/data/external/7',
 '/workspace/data/external/3',
 '/workspace/data/external/4',
 '/workspace/data/external/5',
 '/workspace/data/external/8',
 '/workspace/data/external/6',
 '/workspace/data/external/10',
 '/workspace/data/external/9',
 '/workspace/data/external/12',
 '/workspace/data/external/1',
 '/workspace/data/external/11']

In [7]:
# Creating Processor
data_1 = '/workspace/data/external/10'

In [8]:
transcript_json = data_1 + "/" +  "data.json"
print(transcript_json)
data = pd.read_json(transcript_json)

/workspace/data/external/10/data.json


In [9]:
data['audioFilename']

0          0_15_NSD-Nepali-Nepali-1925-1935-2020742022.wav
1          0_1_NSD-Nepali-Nepali-1925-1935-20195719500.wav
2          0_2_NSD-Nepali-Nepali-1925-1935-20211519542.wav
3          0_4_NSD-Nepali-Nepali-1925-1935-20183519429.wav
4          0_6_NSD-Nepali-Nepali-1925-1935-20187220332.wav
                               ...                        
19995    3_2_Regional-Guwahati-Nepali-1515-201972615414...
19996    32_Regional-Guwahati-Nepali-1515-2021112161233...
19997    3_2_Regional-Kurseong-Nepali-1850-201810111941...
19998    32_Regional-Guwahati-Nepali-1515-2021119152822...
19999    3_2_Regional-Guwahati-Nepali-1515-202029152922...
Name: audioFilename, Length: 20000, dtype: object

In [10]:
for i in range(len(data)):
    file = data_1  + "/" + data['audioFilename'][i]
    data['audioFilename'][i] = file

In [11]:
data['audioFilename']

0        /workspace/data/external/10/0_15_NSD-Nepali-Ne...
1        /workspace/data/external/10/0_1_NSD-Nepali-Nep...
2        /workspace/data/external/10/0_2_NSD-Nepali-Nep...
3        /workspace/data/external/10/0_4_NSD-Nepali-Nep...
4        /workspace/data/external/10/0_6_NSD-Nepali-Nep...
                               ...                        
19995    /workspace/data/external/10/3_2_Regional-Guwah...
19996    /workspace/data/external/10/32_Regional-Guwaha...
19997    /workspace/data/external/10/3_2_Regional-Kurse...
19998    /workspace/data/external/10/32_Regional-Guwaha...
19999    /workspace/data/external/10/3_2_Regional-Guwah...
Name: audioFilename, Length: 20000, dtype: object

In [12]:
data.rename(columns = {'text':'sentence'}, inplace = True)
data.rename(columns = {'audioFilename':'path'}, inplace = True)

In [13]:
len(data)

20000

In [14]:
train = data[0:10000]
len(train)

10000

In [15]:
common_voice_train_1 = Dataset.from_pandas(train[0:5000])
common_voice_train_2 = Dataset.from_pandas(train[5000:])

print(len(common_voice_train_1))
print(len(common_voice_train_2))

5000
5000


In [16]:
print(common_voice_train_1['sentence'][4])
print(common_voice_train_2['sentence'][4])

शान्ति र समृद्धिको उपलब्धिहरू का लागि महत्त्वपूर्ण भूमिका खेल्न सक्छ जसद्वारा वैश्विक शान्ति समृद्धि हासिल गर्न सकिन्छ
मिलियन सँग भेटबाट गर्नुहुनेछ यस भ्रमण ले दुई देश बीच विद्यमान दुई पक्ष सम्बन्धको समीक्षाका अवसर प्रदान गर्ने साथै परस्पर हितका क्षेत्रहरूमा सहयोग बनाउने अवसर प्रदान गर्नेछ


In [17]:
common_voice_train_1 = common_voice_train_1.remove_columns([ "collectionSource","snr",  "duration", "gender"])
common_voice_train_2 = common_voice_train_2.remove_columns(["collectionSource","snr",  "duration", "gender"])

In [18]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [19]:
common_voice_train_1 = common_voice_train_1.map(remove_special_characters)
common_voice_train_2 = common_voice_train_2.map(remove_special_characters)

In [20]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [21]:
common_voice_train_1 = common_voice_train_1.map(speech_file_to_array_fn, remove_columns=common_voice_train_1.column_names,num_proc=4)

RuntimeError: Error opening '/workspace/data/external/10/0_11_NSD-Nepali-Nepali-1925-1935-2019916194720.wav': File contains data in an unknown format.

In [ ]:
common_voice_train_2 = common_voice_train_2.map(speech_file_to_array_fn, remove_columns=common_voice_train_2.column_names,num_proc=4)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train_1)-1)

ipd.Audio(data=np.asarray(common_voice_train_1[rand_int]["speech"]), autoplay=True, rate=16000)

In [22]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [25]:
tokenizer = Wav2Vec2CTCTokenizer("/workspace/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [26]:
len(processor.tokenizer)

62

In [27]:
common_voice_train_1 = common_voice_train_1.map(prepare_dataset, remove_columns=common_voice_train_1.column_names, batch_size=8, num_proc=4, batched=True)

In [28]:
common_voice_train_2 = common_voice_train_2.map(prepare_dataset, remove_columns=common_voice_train_2.column_names, batch_size=8, num_proc=4, batched=True)

In [29]:
common_voice_train_2

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1069
})

In [30]:
common_voice_train = concatenate_datasets([common_voice_train_1, common_voice_train_2])

In [31]:
processed_data_dir_path_ = processed_data_dir_path + "/" + str(29)

In [32]:
processed_data_dir_path_

'/workspace/data_processed/29'

In [33]:
common_voice_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 2069
})

In [34]:
common_voice_train.save_to_disk(processed_data_dir_path_)